In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType, FloatType
import pyspark.sql.functions as F

### 1. Brands Dimension Table 

In [0]:
catalog_name = 'ecommerce'

# Define the schema for the data file 
brand_schema = StructType([
    StructField('brand_code',StringType(),False),
    StructField('brand_name',StringType(),True),
    StructField('category_code',StringType(),True),
])

In [0]:
raw_data_path = "/Volumes/ecommerce/source_data/raw/brands/*.csv"

df = spark.read.option('header','true').option('delimiter',',').schema(brand_schema).csv(raw_data_path)


# Add metadata column 
df = df.withColumn('_source_file', F.col('_metadata.file_path')) \
        .withColumn('ingested_at', F.current_timestamp())

display(df.limit(5))

In [0]:
# Create delta table 

df.write.mode('delta') \
    .mode('overwrite') \
    .option('mergeSchema','true') \
    .saveAsTable(f'{catalog_name}.bronze.brz_brands')